In [1]:
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import imutils
import cv2
import time
import os
from tqdm import tqdm
import tensorflow as tf
import pandas as pd

In [2]:
# physical_devices = tf.config.list_physical_devices('GPU')
# if physical_devices:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
#     tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')

In [3]:
prototxtPath = r"D:\sentirobots\code\fRojas\deploy.prototxt.txt"
weightsPath = r"D:\sentirobots\code\fRojas\res10_300x300_ssd_iter_140000.caffemodel"

net = cv2.dnn.readNet(prototxtPath, weightsPath)

In [4]:
model=tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',input_shape=(224, 224, 3))

In [5]:
from tensorflow.keras.layers import Dense, Dropout, Flatten
import tensorflow as tf

nb_class = 6
hidden_dim = 4096

inputs = tf.keras.Input(shape=(224, 224, 3))

custom_vgg_model=tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',input_shape=(224, 224, 3))

last_layer = custom_vgg_model.get_layer('block5_pool').output
x = tf.keras.layers.Flatten(name='flatten')(last_layer)
x = tf.keras.layers.Dense(hidden_dim, activation=tf.nn.relu, name='fc6')(x)
# x = tf.keras.layers.Dense(hidden_dim, activation=tf.nn.relu, name='fc6', kernel_regularizer='l1')(x)
x = tf.keras.layers.Dropout(.3)(x)
x = tf.keras.layers.Dense(hidden_dim, activation=tf.nn.relu, name='fc7')(x)
x = tf.keras.layers.Dropout(.3)(x)
x = tf.keras.layers.Dense(nb_class, activation=tf.nn.softmax, name='fc8')(x)
custom_vgg_model = tf.keras.Model(custom_vgg_model.input, x)
custom_vgg_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
from tensorflow.keras.models import load_model
custom_vgg_model.load_weights(r'D:\sentirobots\code\fRojas\weigth_friend_V5.h5')
print("Loaded model from disk")

Loaded model from disk


In [7]:
import matplotlib.pyplot as plt
import cv2

In [8]:
#!mkdir '/content/drive/MyDrive/Meld/MELD.Raw/Train cropped images'   #Primera ejecución
#!mkdir 'D:/sentirobots/Datasets/MELD/Caras MELD/Dev Actors images'    #Segunda ejecución

In [9]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4979235863030735314
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2914307278
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16537782741570870233
physical_device_desc: "device: 0, name: Quadro P1000, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [10]:
import cv2
import os
import numpy as np
import pandas as pd

CATEGORIES = ["Chandler", "Joey", "Monica", "Phoebe", "Rachel", "Ross"]
UNKNOWN_CATEGORY = "Extras" 

df = pd.read_csv(r'D:\sentirobots\Datasets\seguir_proceso_MELD_cropped.csv')
df

,File Name,Width_x,Height_x,Full Path_x,Width_y,Height_y,Full Path_y
0,result_dia0_utt0.0,NaN,NaN,NaN,79,79,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...
1,result_dia0_utt0.1,NaN,NaN,NaN,79,79,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...
2,result_dia0_utt0.10,NaN,NaN,NaN,81,81,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...
3,result_dia0_utt0.100,NaN,NaN,NaN,74,74,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...
4,result_dia0_utt0.101,NaN,NaN,NaN,77,77,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...
...,...,...,...,...,...,...,...
406469,result_dia9_utt7.55,NaN,NaN,NaN,168,168,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...
406470,result_dia9_utt7.80,NaN,NaN,NaN,194,194,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...
406471,result_dia9_utt7.81,NaN,NaN,NaN,187,187,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...
406472,result_dia9_utt7.82,NaN,NaN,NaN,184,184,D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN ...


In [11]:
output_images_path = r"D:\sentirobots\Datasets\MELD\Caras MELD\TRAIN imagenes actores"

for _, row in df.iterrows():
    img_path = row['Full Path_y']
    frame = cv2.imread(img_path)
    blob = cv2.dnn.blobFromImage(frame, 1, (224, 224))
    net.setInput(blob)
    out = net.forward()

    is_actor_detected = False

    for i in range(0, out.shape[2]):
        if out[0, 0, i, 2] > 0.5:
            box = out[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (Xi, Yi, Xf, Yf) = box.astype("int")
            Xi = max(0, Xi)
            Yi = max(0, Yi)
            Xf = min(frame.shape[1], Xf)
            Yf = min(frame.shape[0], Yf)

            face = frame[Yi:Yf, Xi:Xf]

            if Yf - Yi > 10 and Xf - Xi > 10:
                face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_AREA)
                xt = np.asarray(face)
                xt = preprocess_input(xt)
                xt = np.expand_dims(xt, axis=0)
                preds = custom_vgg_model.predict(xt)

                if np.amax(preds) > 0.9:
                    actor = str(CATEGORIES[np.argmax(preds)])
                    path1 = os.path.join(output_images_path, actor)
                    if not os.path.exists(path1):
                        os.makedirs(path1)
                    dst = os.path.join(path1, os.path.basename(img_path))
                    cv2.imwrite(dst, frame[Yi + 1:Yf, Xi + 1:Xf])
                    is_actor_detected = True
                    break

    if not is_actor_detected:
        unknown_path = os.path.join(output_images_path, UNKNOWN_CATEGORY)
        if not os.path.exists(unknown_path):
            os.makedirs(unknown_path)
        dst = os.path.join(unknown_path, os.path.basename(img_path))
        cv2.imwrite(dst, frame)


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 22ms/step


KeyboardInterrupt: 